<a href="https://colab.research.google.com/github/mearsdj/Amazon_Vine_Analysis/blob/main/GoogleColabLessonPractice1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://ppa.lau

In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [6]:
dataframe = spark.createDataFrame([
                                    (0,"making a flipping df"),
                                   (1,"from flipping scratch"),
                                   (2,"how flipping great!")
                                   ],['id',"words"]
                                   )
dataframe.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|making a flipping df|
|  1|from flipping scr...|
|  2| how flipping great!|
+---+--------------------+



In [8]:
#REad data from csv
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_16/food.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('food.csv'),sep=",",header=True)
df.show()

+-------+-----+
|   food|price|
+-------+-----+
|  pizza|    0|
|  sushi|   12|
|chinese|   10|
+-------+-----+



In [9]:
#print dataframe scheme
df.printSchema()

root
 |-- food: string (nullable = true)
 |-- price: string (nullable = true)



In [10]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType


In [12]:
schema = [StructField("food",StringType(),True),StructField("price",IntegerType(),True),]

In [14]:
schema

[StructField(food,StringType,true), StructField(price,IntegerType,true)]

In [15]:
#pass in our fields to final schema:
final = StructType(fields=schema)
final

StructType(List(StructField(food,StringType,true),StructField(price,IntegerType,true)))

In [16]:
#read in data again, but now with your predefined schema:
dataframe = spark.read.csv(SparkFiles.get('food.csv'),sep=",",header=True, schema=final)
dataframe.printSchema()

root
 |-- food: string (nullable = true)
 |-- price: integer (nullable = true)



In [17]:
dataframe['price']

Column<'price'>

In [18]:
dataframe.select('price')

DataFrame[price: int]

In [19]:
type(dataframe['price'])

pyspark.sql.column.Column

In [20]:
type(dataframe.select('price'))

pyspark.sql.dataframe.DataFrame

In [21]:
# Add new column
dataframe.withColumn('newprice', dataframe['price']).show()
# Update column name
dataframe.withColumnRenamed('price','newerprice').show()

+-------+-----+--------+
|   food|price|newprice|
+-------+-----+--------+
|  pizza|    0|       0|
|  sushi|   12|      12|
|chinese|   10|      10|
+-------+-----+--------+

+-------+----------+
|   food|newerprice|
+-------+----------+
|  pizza|         0|
|  sushi|        12|
|chinese|        10|
+-------+----------+



In [23]:
#double the price
dataframe.withColumn("price",dataframe["price"]*2).show()
#add a dollar to price
dataframe.withColumn("price",dataframe['price']+1).show()

# half px
dataframe.withColumn("price",dataframe['price'] * 0.5).show()

+-------+-----+
|   food|price|
+-------+-----+
|  pizza|    0|
|  sushi|   24|
|chinese|   20|
+-------+-----+

+-------+-----+
|   food|price|
+-------+-----+
|  pizza|    1|
|  sushi|   13|
|chinese|   11|
+-------+-----+

+-------+-----+
|   food|price|
+-------+-----+
|  pizza|  0.0|
|  sushi|  6.0|
|chinese|  5.0|
+-------+-----+

